In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import math
import copy
import datetime
from ipynb.fs.defs.Neural_Network_Module import Dataset, transformations, train_nn, prediction
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
def plot_error_dist(actual, predictions, model_type, case=''):
    """Prints the error distribution plot
    Parameters
    --------
    actual : pd.Series
        A Series with the actual class of each prediction
        
    predictions : pd.Series
        A Series with the predicted class of each prediction
        
    model_type : str
        Could be 'class_regression' or 'mosquito_regression' or 'classification'
    
    case: str, optional
        Title of the plot (default= '')
    """
    error = np.abs(actual-predictions).tolist()
    if model_type != 'mosquito_regression':
        bins = np.arange(len(actual.unique())) - 0.5
        plt.hist(error, bins)
        plt.xticks(range(len(actual.unique())))
    else:
        plt.hist(error)
    plt.xlabel('abs(error)')
    plt.title('Error Distribution \n' + case)
    plt.show()

In [6]:
def plot_hist(actual, predictions, model_type, case=''):
    """Prints the histogram of the actual values and the predicted values
    Parameters
    --------
    actual : pd.Series
        A Series with the actual class of each prediction
        
    predictions : pd.Series
        A Series with the predicted class of each prediction
        
    model_type : str
        Could be 'class_regression' or 'mosquito_regression' or 'classification'
    
    case: str, optional
        Title of the plot (default= '')
    """
    plt.figure(figsize=(10,8)) 
    if model_type != 'mosquito_regression':
        bins = np.arange(len(actual.unique())+1)-0.5
        plt.hist(actual, bins=bins, alpha=0.5, label='actual')
        plt.hist(predictions, bins=bins, alpha=0.5, label='prediction')
        plt.xticks(range(len(actual.unique())))
    else:
        plt.hist(actual, alpha=0.5, label='actual')
        plt.hist(predictions, alpha=0.5, label='prediction')
    plt.legend()
    plt.title('Histogram of actual vs predicted values \n'+case)
    plt.show()

In [3]:
def plot_error_per_class(test, case=''):
    """Prints the error distribution per class plot
    Parameters
    --------
    actual : lst
        A list with the actual class of each prediction
        
    case: str, optional
        Title of the plot (Area and mosquito genus) (default= '')
    """
    labels = test.loc[:,'actual'].unique().tolist()
    labels.sort()
    f = []
    length = []
    for k in labels:
        cc = test.loc[test['actual']==k]
        length.append(len(cc))
        actual = cc.loc[:,'actual']
        predictions = cc.loc[:,'prediction']
        mae_class = mean_absolute_error(actual, predictions)
        f.append(mae_class)
    labels = [str(int(e)) for e in labels]
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(labels,f)
    for i, v in enumerate(f):
        ax.text(i, v, str('%.2f'%(v)), rotation=30)
        ax.text(i, v/2,'n = '+ str(length[i]), weight="bold", ha='center', rotation=90)
    plt.xlabel('class')
    plt.ylabel('MAE')
    plt.title('MAE per class ' + case)
    plt.show()
    
    print('-----------|class error-MAE| difference-----------')
    z = np.abs(f-mean_absolute_error(actual, predictions))
    print('mean:',z.mean())
    print('std:',z.std())
    print('coefficient of variation (std/mean):',z.std()/z.mean())
    
    print()
    
    print('----------normalized difference-------------')
    min_val = min(z)
    max_val = max(z)
    z = (z - min_val) / (max_val-min_val)
    print('mean:',z.mean())
    print('std:',z.std())

In [4]:
def plot_error_per_month(df, case=''):
    """Prints the error per month
    Parameters
    --------
    df : dataframe
        A dataframe containing the data
    
    case: str, optional
        Title of the plot (Area and mosquito genus) (default= '')
    """
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    labels = (df['dt_prediction'].dt.month).unique()
    labels.sort()
    labels = [str(e) for e in labels]
    df['abs(error)'] = np.abs(df['actual']-df['prediction'])
    f = df.groupby(by=[df['dt_prediction'].dt.month])['abs(error)'].mean().values
    length = df.groupby(by=[df['dt_prediction'].dt.month])['dt_prediction'].count().values
    ax.bar(labels,f)
    for i, v in enumerate(f):
        ax.text(i, v, str('%.2f'%(v)),rotation=30)
        ax.text(i, v/2,'n = '+ str(length[i]),weight="bold",ha='center', rotation=90)
    plt.xlabel('Month')
    plt.ylabel('MAE')
    plt.title('Mean Absolute Error per month ' + case)
    plt.show()

In [5]:
def scatter_plot_error(actual, prediction, case=''):
    """Prints the error in relation with the distance of point from the train region
    Parameters
    --------
    df : dataframe
        A dataframe containing the data
    
    case: str, optional
        Title of the plot (Area and mosquito genus) (default= '')
    """
    # choose the input and output variables
    x, y = actual, np.abs(actual-prediction)
    plt.scatter(x, y)
    plt.xlabel('Mosquito bins')
    plt.ylabel('Error')
    plt.title('Scatterplot of error ' + case)
    plt.show()

In [7]:
def plot_error_per_group(actual,prediction,case=''):
    """Prints the error distribution plot
    Parameters
    --------
    actual : pd.Series
        A Series with the actual class of each prediction
        
    predictions : pd.Series
        A Series with the predicted class of each prediction
        
    case: str, optional
        Title of the plot (default= '')

    """
    test = {'mosq_now':actual,'predictions':prediction}
    test = pd.DataFrame(test)
    test['classes'] = pd.cut(x=test['mosq_now'], bins=[-1, 100, 200, 300, 400, 500, np.inf],
                      labels=['0-100', '101-200', '201-300', '301-400', '401-500', '500<'])
    labels = test['classes'].unique().tolist()
    labels.sort()
    f = []
    length = []
    for k in labels:
        cc = test.loc[test['classes']==k]
        length.append(len(cc))
        actual = cc.loc[:,'mosq_now']
        predictions = cc.loc[:,'predictions']
        f.append(mean_absolute_error(actual, predictions))
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar(labels,f)
    for i, v in enumerate(f):
        ax.text(i, v, str('%.2f'%(v)),rotation=30)
        ax.text(i, v/2,'n = '+ str(length[i]),weight="bold",ha='center', rotation=90)
    plt.xlabel('Mosquito Group')
    plt.ylabel('MAE')
    plt.title('MAE per Mosquito group \n'+case)
    plt.show()

In [8]:
def error_cdf(actual,prediction, case=''):
    """Prints the cdf of errors
    Parameters
    --------
    actual : pd.Series
        A Series with the actual class of each prediction
        
    predictions : pd.Series
        A Series with the predicted class of each prediction
        
    case: str, optional
        Title of the plot (default= '')

    """
    error = np.abs(actual-prediction)
    
    a = np.sort(error.unique())
    b = np.array(error)
    cdf = np.zeros(len(a))
    for k, val in enumerate(a):
        mask_d = b <= val
        cdf[k] = mask_d.sum()/ len(b)
    plt.figure(figsize=(8,8))
    plt.plot(a,cdf)
    plt.grid()
    plt.xlabel('abs(error)',fontsize=18)
    plt.ylabel('CDF',fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    #plt.title('CDF of error \n' + case)
    plt.show() 
    
    b = np.sort(error)
    a = np.arange(1,len(error)+1) 
    cdf = np.zeros(len(a))
    for k, val in enumerate(b):
        cdf[k] = b[k]
    plt.plot(a,cdf)
    plt.grid()
    plt.xlabel('Number of samples',fontsize=18)
    plt.ylabel('Error',fontsize=18)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.title('CDF of error \n' + case)
    plt.show()

In [9]:
def metrics(train, test, threshold=3):
    """Calculates the perfomance of the model on train and test set
    Parameters
    --------
    train : Dataframe
        A Dataframe with the actual and the predicted values on the train set
        
    test : Dataframe
        A Dataframe with the actual and the predicted values on the train set
        
    threshold: int, optional
        A threshold to calculate percentage of error < threshold (default= 3)

    """    
    print('MAE on train set: ', mean_absolute_error(train['actual'], train['prediction']))

    print('min prediction:',min(train['prediction']))
    print('max prediction:',max(train['prediction']))
    
    print()

    print('MAE on test set: ', mean_absolute_error(test['actual'], test['prediction']))
    perc = ((np.abs(test['actual']-test['prediction']) < (threshold+0.5)).mean())*100
    print('Error <= '+str(threshold)+':',"%.2f"%perc,'%')

    print('min prediction:',min(test['prediction']))
    print('max prediction:',max(test['prediction']))

In [10]:
def validation(train, test, model_type, case=''):
    """Prints plots about the performance of the model on the test set
    
    Parameters
    --------        
    test : Dataframe
        A Dataframe with the actual and the predicted values on the train set
    
    model_type : str
        Could be 'class_regression' or 'mosquito_regression' or 'classification'
        
    case: str, optional
        Title of the plot (default= '')
    """
    if model_type != 'mosquito_regression':
        metrics(train, test)
        plot_error_per_class(test, case)
    else:
        metrics(train, test, threshold=30)
        plot_error_per_group(test['actual'],test['prediction'], case)
        error_cdf(test['actual'],test['prediction'], case)
    scatter_plot_error(test['actual'],test['prediction'], case)
    plot_error_dist(test['actual'],test['prediction'], model_type, case)
    plot_hist(test['actual'],test['prediction'], model_type, case)
    plot_error_per_month(test, case)

In [ ]:
def test_model_random_split(data, model, test=None, filepath = '', date_col = 'dt_placement',
                            fi=False, case='',export=False):
    """Trains a model on random splitted data
    
    Parameters
    --------
    data : dataframe
        A dataframe containing the data
    
    model : torch.nn.model
        A NN model to train
        
    embedding_data : dataframe, optional
        A datafrane with the categorical features (default = None)
        
    filepath : srt, optional
        The path of the file to export the results (default = '')
        
    date_col : str, optional
        The name of the date column (default = 'dt_placement')
        
    fi: boolean, optional
        If True, prints the feature importance of the model (time consuming process) (default = False)
    
    case : str, optional
        The title of case for the plot (default='')
    
    export : boolean, optional
        Export a csv with the feature importance and a csv with the test data (default = False)
    """
    case = case + " random validation"
    
    if test is None:
        np.random.seed(1)
        drop_index = np.random.randint(0, len(data), int((len(data)*20)/100))
        test = data.iloc[drop_index,:].reset_index(drop=True)
        train = data.drop(drop_index).reset_index(drop=True)
#     if test is None:
#         train, test = train_test_split(data, test_size=0.2, random_state=3)
#         train = train.reset_index(drop=True)
#         test = test.reset_index(drop=True)
    
    del train[date_col]
    date = test[date_col]
    del test[date_col]
    
    mosq_col = test.columns[-1]
    max_val= train.iloc[:,-1].max()
    
    model_int = copy.deepcopy(model)
    
    features = None
    if fi:
        if model.embedding_data is not None:
            columns = data.drop(columns=model.embedded_data.columns.tolist()).columns[:-1].tolist()
            columns = columns+embedded_data.columns.tolist()
            features = columns
        else:
            features = data.columns[:-1].tolist()
    
    train_X, train_y, test_X, test_y = transformations(train, test = test, model_type = model.model_type, 
                                                       embedding_data = model.embedding_data,
                                                       transformation_list = model.transformation_list)

    training_set = Dataset(train_X, train_y)

    testing_set = Dataset(test_X, test_y)

    results_train, results_test, _ = train_nn(model = model_int, train_set = training_set, test_set = testing_set,
                               features=features, max_val = max_val)
    

    test[date_col] = date
    test['dt_prediction'] = test[date_col] + datetime.timedelta(days=15)
    test['prediction'] = results_test['prediction']
    test.loc[test['prediction']<0,'prediction'] = 0
    test['error'] = test[mosq_col] - test['prediction']
    test['abs(error)'] = np.abs(test[mosq_col] - test['prediction'])
    
    test = test.rename(columns={mosq_col:'actual'})

    validation(results_train, test, model_type=model.model_type, case=case)  
    
    if export:
        csv = filepath + case + '.csv'
        test.to_csv(csv,index=False)

In [ ]:
def train_model_KFold(data, model, cv=10, date_col='dt_placement', case=''):
    """Trains a model on KFold splitted data
    
    Parameters
    --------
    data : dataframe
        A dataframe containing the data
    
    model : torch.nn.model
        A NN model to train
        
    date_col : str, optional
        The name of the date column (default = 'dt_placement')
    """
    
    case = case + ' ' +str(cv) +" fold validation"
    kf = KFold(n_splits=cv, random_state=0, shuffle=True)
    
    df_train = pd.DataFrame() 
    df_test = pd.DataFrame()
    
    for train_index, test_index in kf.split(data):
        
        model_int = copy.deepcopy(model)
        
        train = data.iloc[train_index,:].reset_index(drop=True)        
        test = data.iloc[test_index,:].reset_index(drop=True)
        
        del train[date_col]
        date = test[date_col]
        del test[date_col]
        

        train_X, train_y, test_X, test_y = transformations(train, test = test, model_type = model.model_type, 
                                                           embedding_data = model.embedding_data,
                                                           transformation_list = model.transformation_list)

        training_set = Dataset(train_X, train_y)

        testing_set = Dataset(test_X, test_y)

        results_train, results_test, _ = train_nn(model = model_int, train_set = training_set, test_set = testing_set,
                                   max_val = train.iloc[:,-1].max())
                
        results_test['dt_prediction'] = date + datetime.timedelta(days=15)
        
        df_train = pd.concat([df_train, results_train])
        df_test = pd.concat([df_test, results_test])
        
     
    validation(df_train, df_test, model_type=model.model_type, case=case)

In [ ]:
def operational_validation(data, model, date, filepath ='', date_col='dt_placement', case='', fi=False, export=False):
    """Trains a model on data of the previous months and evaluates on data of the next month iteratively.
    
    Parameters
    --------
     data : dataframe
        A dataframe containing the data
    
    model : torch.nn.model
        A NN model to train
        
    date: str
        The date to start the testing process from (format: YYYY-MM-DD)
    
    embedding_data : dataframe, optional
        A datafrane with the categorical features (default = None)
    
    filepath : srt
        The path of the file to export results (default ='')
        
    date_col : str, optional
        The name of the date column (default = 'dt_placement')
    
    case : str, optional
        The title of case for the plot (default ='')
    
    export : boolean, optional
        Export a csv with the feature importance and a csv with the test data (default=False)
        
    Raise
    --------------
    ValueError
        If date > maximum available date in the dataset
        
    """
    data = data.copy()
    if (pd.to_datetime(date) > data['dt_placement'].max()):
        raise ValueError('date argument given must be before than '+ str(data['dt_placement'].max()))
        
    mosq_col = data.columns[-1]
        
    case = case + " operational validation"
    df = pd.DataFrame()
    
    df_train = pd.DataFrame() 
    df_test = pd.DataFrame() 
    
    months = data.loc[data['dt_placement']>pd.to_datetime(date),'dt_placement'].dt.to_period('M').unique()
    months = months.strftime('%Y-%m')
    months.sort()
    
    features = None
    if fi:
        if model.embedding_data is not None:
            columns = data.drop(columns=model.embedded_data.columns.tolist()+[date_col]).columns[:-1].tolist()
            columns = columns+embedded_data.columns.tolist()
            features = columns
        else:
            features = data.columns[:-1].tolist()
            
    model_int = copy.deepcopy(model)
        
    for i in months:
        date1 = i +'-01'
        if i.split('-')[1]>='09':
            if i.split('-')[1]=='12':
                date2 = str(int(i.split('-')[0])+1) +'-01-01'
            else:
                date2 = i.split('-')[0] + '-' + str(int(i.split('-')[1]) + 1) + '-01'
        else:
            date2 = i.split('-')[0] + '-0' + str(int(i.split('-')[1]) + 1) + '-01'

        train = data.loc[data[date_col] < pd.to_datetime(date1)]
        train = train.reset_index(drop=True)
        del train[date_col]
        
        test = data.loc[data[date_col] >= pd.to_datetime(date1)]
        test = test.loc[test[date_col] < pd.to_datetime(date2)]
        test = test.reset_index(drop=True)
        date = test[date_col]
        del test[date_col]
        
        train_X, train_y, test_X, test_y = transformations(train, test = test, model_type = model.model_type,
                                                           embedding_data = model.embedding_data)

        training_set = Dataset(train_X, train_y)
        
        testing_set = Dataset(test_X, test_y)

        results_train, results_test, _ = train_nn(model = model_int, train_set = training_set, test_set = testing_set,
                                                  features=features, max_val = train.iloc[:,-1].max())
        
        test[date_col] = date
        test['dt_prediction'] = test[date_col] + datetime.timedelta(days=15)
        test['prediction'] = results_test['prediction']
        test.loc[test['prediction']<0,'prediction'] = 0
        test['error'] = test[mosq_col] - test['prediction']
        test['abs(error)'] = np.abs(test[mosq_col] - test['prediction'])
        
        df_train = pd.concat([df_train,results_train])
        df_test = pd.concat([df_test,test])
        
    df_test = df_test.rename(columns={mosq_col:'actual'})
    
    years = df_test['dt_prediction'].dt.to_period('Y').unique()
    years = years.strftime('%Y')
    years.sort()
    for i in years:
        print('====================')
        print(i)
        print('====================')
        df_test_year = df_test.loc[df_test['dt_prediction'].dt.year==int(i)].reset_index(drop=True)
        validation(df_train, df_test_year, model_type=model.model_type, case=case)
    
    df_test = df_test.rename(columns={'actual':mosq_col})
    
    if export:
        csv = filepath + case + '.csv'
        df_test.to_csv(csv,index=False)

In [ ]:
def predict(data, test, model, filepath = '', date_col = 'dt_placement', case='', fi = False, export=False):
    """Trains a model on random splitted data
    
    Parameters
    --------
    data : dataframe
        A dataframe containing the data
        
    embedding_data : dataframe, optional
        A datafrane with the categorical features (default = None)
        
    filepath : srt, optional
        The path of the file to export the results (default = '')
        
    date_col : str, optional
        The name of the date column (default = 'dt_placement')
    
    case : str, optional
        The title of case for the plot (default='')
    
    export : boolean, optional
        Export a csv with the feature importance and a csv with the test data (default = False)
    """
    case = case + " predict"
    
    max_val= data.iloc[:,-1].max()
    model_int = copy.deepcopy(model)
    
    del data[date_col]
    date = test[date_col]
    del test[date_col]
    
    features = None
    if fi:
        if model.embedding_data is not None:
            columns = data.drop(columns=model.embedded_data.columns.tolist()+[date_col]).columns[:-1].tolist()
            columns = columns+embedded_data.columns.tolist()
            features = columns
        else:
            features = data.columns[:-1].tolist()
    
    train_X, train_y, test_X, test_y = transformations(train, test = test, model_type = model.model_type,
                                                       embedding_data = model.embedding_data)

    training_set = Dataset(train_X, train_y)

    testing_set = Dataset(test_X, test_y)
    
    results_test = prediction(model_int, train_set, test_set, max_val = max_val, features = features)
 
    
    test[date_col] = date
    test['dt_prediction'] = test[date_col] + datetime.timedelta(days=15)
    test['prediction'] = results_test['prediction']
    
    if export:
        csv = filepath + case + '.csv'
        test.to_csv(csv,index=False)
    return test